# Surebets

Situation where you can profit regardless of a match outcome based on different bookmakers providing different odds

If (1/odds1) + (1/odds2) <1.0, you've got a sure bet


| Bookie | Over     | Under     |
|--------|----------|-----------|
|   1    |   **2**  |   1.9     |
|   2    |  1.8     |  **2.1**  |


With $100 to bet. $51.21 on the over with bookie 1, $48.78 on the under with bookie 2

**Scenario 1 - Over**
You win on Bookmaker 1 but lose on Bookmaker 2
`Profit 1= 51.21*2 — 51.21 — 48.78 = 2.43`

**Scenario 2 - Under**
You win on Bookmaker 2 but lose on Bookmaker 1
`Profit 2 = 48.78*2.1 — 51.21 — 48.78= 2.44`

In [40]:
from selenium import webdriver
import pandas as pd
import time

In [41]:
def find_surebet(odds1, odds2):
    sum_odds = (1/odds1) + (1/odds2)
    if sum_odds<1:
        message = 'Surebet Found!'
    else:
        message = 'Keep scraping'
    return message

In [12]:
!xattr -d com.apple.quarantine /Users/jaredgodar/codeup-data-science/surebet/chromedriver


In [42]:
#Writing our First Selenium Python Test
web = 'https://sports.tipico.de/en/all/football/spain/la-liga' #you can choose any other league (update 1)
path =  '/Users/jaredgodar/codeup-data-science/surebet/chromedriver'
driver = webdriver.Chrome(path)
driver.get(web)


/var/folders/9y/17ptyrds267724n2_7s7hx340000gn/T/ipykernel_30941/1413878318.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [19]:
#Make ChromeDriver click a button
#time.sleep(5) #add implicit wait, if necessary
#accept = driver.find_element_by_xpath('//*[@id="_evidon-accept-button"]')
#accept.click()

In [43]:
#Initialize  storage
teams = []
x12 = [] #3-way
odds_events = []

In [16]:
#select only upcoming matches box
box = driver.find_element_by_xpath('//div[contains(@testid, "Program_SELECTION")]') #update 3
#Looking for 'sports titles'
sport_title = box.find_elements_by_class_name('SportTitle-styles-sport')

/var/folders/9y/17ptyrds267724n2_7s7hx340000gn/T/ipykernel_30941/2537215173.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  box = driver.find_element_by_xpath('//div[contains(@testid, "Program_SELECTION")]') #update 3
/opt/homebrew/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:464: UserWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  warnings.warn("find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead")


In [45]:
for sport in sport_title:
    # selecting only football
    if sport.text == 'Football':
        parent = sport.find_element_by_xpath('./..') #immediate parent node
        grandparent = parent.find_element_by_xpath('./..') #grandparent node = the whole 'football' section
        #Looking for single row events
        single_row_events = grandparent.find_elements_by_class_name('EventRow-styles-event-row')
        #Getting data
        for match in single_row_events:
            #'odd_events'
            odds_event = match.find_elements_by_class_name('EventOddGroup-styles-odd-groups')
            odds_events.append(odds_event)
            # Team names
            for team in match.find_elements_by_class_name('EventTeams-styles-titles'):
                teams.append(team.text)
        #Getting data: the odds        
        for odds_event in odds_events:
            for n, box in enumerate(odds_event):
                rows = box.find_elements_by_xpath('.//*')
                if n == 0:
                    x12.append(rows[0].text)

driver.quit()
#Storing lists within dictionary
dict_gambling = {'Teams': teams, '1x2': x12}
#Presenting data in dataframe
df_gambling = pd.DataFrame.from_dict(dict_gambling)
print(df_gambling)

TypeError: 'WebElement' object is not iterable

In [18]:
df_gambling

,Teams,1x2
0,Real Betis\nAthletic Bilbao,2.40\n3.00\n3.00
1,Real Sociedad\nCD Alaves,1.53\n3.80\n7.00
2,FC Barcelona\nCA Osasuna,1.32\n5.30\n8.50
3,RCD Mallorca\nReal Madrid,6.00\n4.30\n1.52
4,Athletic Bilbao\nCF Getafe,1.70\n3.50\n5.50
5,CD Alaves\nGranada CF,2.10\n3.30\n3.60
6,Elche CF\nCF Valencia,3.00\n3.20\n2.40
7,CA Osasuna\nUD Levante,2.00\n3.40\n3.70
8,Rayo Vallecano\nAtletico Madrid,3.70\n3.40\n2.00
9,Espanyol Barcelona\nRCD Mallorca,2.20\n3.30\n3.30


---

[Source](https://medium.datadriveninvestor.com/make-money-with-python-the-sports-arbitrage-project-3b09d81a0098)

In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [23]:
#changing chromedriver default options
options = Options()
options.headless = True
options.add_argument('window-size=1920x1080') #Headless = True

web = 'https://sports.tipico.de/en/live/soccer'
path =  '/Users/jaredgodar/codeup-data-science/surebet/chromedriver'

#execute chromedriver with edited options
driver = webdriver.Chrome(path, options=options)
driver.get(web)

/var/folders/9y/17ptyrds267724n2_7s7hx340000gn/T/ipykernel_30941/3841745913.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path, options=options)


- `options = Options()`creates an instance of the Options class
- `options.headless = True` turns on the headless mode
- `options.add_argument(‘window-size=1920x1080’)` opens the windows in a customized size on the background
- `webdriver.Chrome(path, options = options)` applies the changes in the chromedriver
- `driver.get` opens the browser
- `web` represents the betting site’s URL, while `path` represents the chromedriver’s path in your computer

### Select value from dropdown menu

We need to select the betting markets we want to get data from. We do this with `Select` and the Xpath of the dropdown. 
Inspect elements right-click and copy xpath

But first, keep in mind that every time you open some betting sites, a cookies banner is displayed. We need to get rid of them by clicking the ‘accept button.’

To click on the ‘accept button’ and then select on the dropdown menus, we write the following code:

In [24]:
#Make ChromeDriver click a button
#option 1
accept = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="_evidon-accept-button"]')))
#option 2
# time.sleep(2)
# accept = driver.find_element_by_xpath('//*[@id="_evidon-accept-button"]')

#Initialize your storage
teams = []
x12 = []
btts = []
over_under = []
odds_events = []

#select values from dropdown (update 1)
dropdowns = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'SportHeader-styles-drop-down')))

first_dropdown = Select(dropdowns[0])
second_dropdown = Select(dropdowns[1])
third_dropdown = Select(dropdowns[2])

first_dropdown.select_by_visible_text('Both Teams to Score') #update 'Both teams to score?' -> 'Both Teams to Score'
second_dropdown.select_by_visible_text('Over/Under')
third_dropdown.select_by_visible_text('3-Way')

- `WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="_evidon-accept-button"]')))`makes the driver wait until the ‘ok button’ of the cookie banner is clickable. If this throws an error, then use ‘option 2’ instead. To get the Xpath, right-click on the ‘ok button’ and inspect as we did before for the dropdown menu.
- `WebDriverWait(driver,5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '...')))` makes the driver wait some seconds until the dropdown menus are located
- `Select('...')` selects `dropdowns` menus
- `first_dropdown` one of the three dropdowns that contain betting markets
- `first_dropdown.select_by_visible_text()` selects an element inside the dropdown menu with the help of the betting market’s name.

### Scrape live events

In [34]:
#Getting data
for match in single_row_events:
    #'odd_events'
    odds_event = match.find_elements_by_class_name('EventOddGroup-styles-odd-groups')
    odds_events.append(odds_event)
    # Team names
    for team in match.find_elements_by_class_name('EventTeams-styles-titles'):
        teams.append(team.text)
#Getting data: the odds        
for odds_event in odds_events:
    for n, box in enumerate(odds_event):
        rows = box.find_elements_by_xpath('.//*')
        if n == 0:
            x12.append(rows[0].text)
        #5 over/under + goal line
        if n == 1:
            parent = box.find_element_by_xpath('./..')
            goals = parent.find_element_by_class_name('EventOddGroup-styles-fixed-param-text').text
            over_under.append(goals+'\n'+rows[0].text)
            #6 both teams to score
        if n == 2:
            btts.append(rows[0].text)

driver.quit()

/opt/homebrew/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:464: UserWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  warnings.warn("find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead")


MaxRetryError: HTTPConnectionPool(host='localhost', port=55428): Max retries exceeded with url: /session/7032a4740b8558b9913db423f7540034/element/0d359c3d-bf30-4aaf-ae1f-928e29d422bf/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f798003b220>: Failed to establish a new connection: [Errno 61] Connection refused'))

---

## Clean Data

In [31]:
import pandas as pd
import pickle


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [32]:
#Storing lists within dictionary
dict_gambling = {'Teams': teams, 'btts': btts,
                 'Over/Under': over_under, '3-way': x12}
#Presenting data in dataframe
df_tipico = pd.DataFrame.from_dict(dict_gambling)
#clean leading and trailing whitespaces of all odds
df_tipico = df_tipico.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [33]:
df_tipico

,Teams,btts,Over/Under,3-way


---

## Bookie 1

In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import pickle

options = Options()
options.headless = True
options.add_argument('window-size=1920x1080') #Headless = True

web = 'https://sports.tipico.de/en/live/soccer'
path =  '/Users/jaredgodar/codeup-data-science/surebet/chromedriver'

driver = webdriver.Chrome(path, options=options)
driver.get(web)
# driver.maximize_window() #Headless = False

#option 1
accept = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="_evidon-accept-button"]')))
#option 2
# time.sleep(2)
# accept = driver.find_element_by_xpath('//*[@id="_evidon-accept-button"]')
accept.click()

teams = []
x12 = []
btts = []
over_under = []
odds_events = []

# ------(use it only if necessary)------
#scroll down to the bottom to load all matches
# driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
# time.sleep(3) #implicit wait to let the page load all the matches
# ---------------------------------------

#select values from dropdown (update 1)
dropdowns = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'SportHeader-styles-drop-down')))

first_dropdown = Select(dropdowns[0])
second_dropdown = Select(dropdowns[1])
third_dropdown = Select(dropdowns[2])

first_dropdown.select_by_visible_text('Both Teams to Score') #update 'Both teams to score?' -> 'Both Teams to Score'
second_dropdown.select_by_visible_text('Over/Under')
third_dropdown.select_by_visible_text('3-Way')

#update 2
box = driver.find_element_by_xpath('//div[contains(@testid, "Program_UPCOMING")]')
#Looking for 'sports titles'
sport_title = box.find_element_by_class_name('SportTitle-styles-sport')

# update 3 (commented code not necesssary anymore)
# for sport in sport_title:
    # selecting only football
#     if sport.text == 'Football':
parent = sport_title.find_element_by_xpath('./..') #immediate parent node
# update 4 (+3 times .find_element_by_xpath('./..'))
grandparent = parent.find_element_by_xpath('./..').find_element_by_xpath('./..').find_element_by_xpath('./..').find_element_by_xpath('./..')
#3. empty groups
try:
    empty_groups = grandparent.find_elements_by_class_name('EventOddGroup-styles-empty-group')
    empty_events = [empty_group.find_element_by_xpath('./..') for empty_group in empty_groups[:]]
except:
    pass
#Single row event
single_row_events = grandparent.find_elements_by_class_name('EventRow-styles-event-row')
#4 remove empty events from single_row_events
try:
    empty_events
    single_row_events = [single_row_event for single_row_event in single_row_events if single_row_event not in empty_events]
except:
    pass
for match in single_row_events:
    odds_event = match.find_elements_by_class_name('EventOddGroup-styles-odd-groups')
    odds_events.append(odds_event)
    # teams
    for team in match.find_elements_by_class_name('EventTeams-styles-titles'):
        teams.append(team.text)
for odds_event in odds_events:
    for n, box in enumerate(odds_event):
        rows = box.find_elements_by_xpath('.//*')
        if n == 0:
            x12.append(rows[0].text)
        #5 over/under + goal line
        if n == 1:
            parent = box.find_element_by_xpath('./..')
            goals = parent.find_element_by_class_name('EventOddGroup-styles-fixed-param-text').text
            over_under.append(goals+'\n'+rows[0].text)
        #6 both teams to score
        if n == 2:
            btts.append(rows[0].text)

driver.quit()

#7 #unlimited columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#Storing lists within dictionary
dict_gambling = {'Teams': teams, 'btts': btts,
                 'Over/Under': over_under, '3-way': x12}
#Presenting data in dataframe
df_tipico = pd.DataFrame.from_dict(dict_gambling)
#clean leading and trailing whitespaces
df_tipico = df_tipico.applymap(lambda x: x.strip() if isinstance(x, str) else x) #14.0\n or 14 or \n14.0

#Save data with Pickle
output = open('df_tipico', 'wb') #don't forget to change name_of_file
pickle.dump(df_tipico, output)
output.close()
print(df_tipico)

/var/folders/9y/17ptyrds267724n2_7s7hx340000gn/T/ipykernel_30941/855511723.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path, options=options)
/var/folders/9y/17ptyrds267724n2_7s7hx340000gn/T/ipykernel_30941/855511723.py:53: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  box = driver.find_element_by_xpath('//div[contains(@testid, "Program_UPCOMING")]')
/opt/homebrew/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:446: UserWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  warnings.warn("find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead")
/opt/homebrew/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:393: UserWarning: find_element_by_xpath is deprecated. Please use f

                                         Teams               btts       Over/Under              3-way
0         Borussia Dortmund\nArminia Bielefeld   1.75\n4.00\n3.50  3.5\n2.25\n1.60  1.27\n6.00\n10.00
1              Eintracht Frankfurt\nVfL Bochum   2.40\n3.70\n2.40  2.5\n1.65\n2.20   1.58\n4.10\n5.80
2                   Arsenal FC\nLeicester City   1.95\n3.80\n2.95  2.5\n1.57\n2.35   1.40\n4.70\n7.50
3                Wisla Krakow SSA\nLech Poznan   2.80\n3.40\n2.05  2.5\n1.80\n1.90   6.80\n4.40\n1.40
4       Virtus Francavilla\nPaganese Cal. 1926   2.55\n3.50\n2.20  2.5\n1.90\n1.80   1.55\n3.50\n5.70
5                          NK Maribor\nNS Mura   2.85\n3.70\n1.95  2.5\n1.85\n1.85   1.70\n3.60\n4.10
6                       US Catanzaro\nSSC Bari   6.20\n3.50\n1.45  2.5\n2.35\n1.52   2.70\n2.50\n2.95
7                    Foggia Calcio\nAZ Picerno   3.20\n3.50\n1.85  2.5\n2.10\n1.65   1.80\n3.10\n4.40
8              SS Juve Stabia\nTurris Neapolis   4.80\n3.40\n1.57  2.5\n2.00\n1.70

---

## Bookie 2

In [ ]:
path =  '/Users/jaredgodar/codeup-data-science/surebet/chromedriver'


In [38]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import pickle
import re

options = Options()
options.headless = True
web = 'https://sports.bwin.com/en/sports/live/football-4?fallback=false'
path =  '/Users/jaredgodar/codeup-data-science/surebet/chromedriver'

options.add_argument('window-size=1920x1080')
driver = webdriver.Chrome(path, options=options)
driver.get(web)
# driver.maximize_window()

teams = []
x12 = []
btts = []
over_under = [] 
odds_events = []

#switching dropdown
#option1
# time.sleep(2)
# dropdown = driver.find_elements_by_css_selector("div.title.multiple")
#option2
dropdown_1 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="main-view"]/ms-live/ms-live-event-list/div/ms-grid/ms-grid-header/div/ms-group-selector[3]/ms-dropdown/div')))
dropdown_1.click()
dropdown_1.find_element_by_xpath('//*[@id="main-view"]/ms-live/ms-live-event-list/div/ms-grid/ms-grid-header/div/ms-group-selector[3]/ms-dropdown/div[2]/div[10]').click()

box = driver.find_element_by_xpath('//ms-grid[contains(@sortingtracking,"Live")]') #livebox
rows = WebDriverWait(box, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'grid-event')))

for row in rows:
    odds = row.find_elements_by_class_name('grid-option-group')
    try:
        empty_events = row.find_elements_by_class_name('empty') #removing empty odds
        odds = [odd for odd in odds if odd not in empty_events]
    except:
        pass
    for n, odd in enumerate(odds[:3]): #only the 3 first dropdowns
        if n==0:
            x12.append(odd.text)
            grandparent = odd.find_element_by_xpath('./..').find_element_by_xpath('./..')
            teams.append(grandparent.find_element_by_class_name('grid-event-name').text)
        if n==1:
            over_under.append(odd.text)
        if n==2:
            btts.append(odd.text)

driver.quit()

#unlimited columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

dict_gambling = {'Teams':teams,'btts': btts,
                 'Over/Under': over_under, '3-way': x12}

df_bwin = pd.DataFrame.from_dict(dict_gambling)
df_bwin['Over/Under'] = df_bwin['Over/Under'].apply(lambda x:re.sub(',', '.', x))
df_bwin = df_bwin.applymap(lambda x: x.strip() if isinstance(x, str) else x)

output = open('df_bwin', 'wb')
pickle.dump(df_bwin, output)
output.close()
print(df_bwin)

/var/folders/9y/17ptyrds267724n2_7s7hx340000gn/T/ipykernel_30941/4064743173.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path, options=options)
/opt/homebrew/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:393: UserWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  warnings.warn("find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main-view"]/ms-live/ms-live-event-list/div/ms-grid/ms-grid-header/div/ms-group-selector[3]/ms-dropdown/div[2]/div[10]"}
  (Session info: headless chrome=99.0.4844.51)
Stacktrace:
0   chromedriver                        0x00000001033ab370 chromedriver + 4420464
1   chromedriver                        0x0000000103342940 chromedriver + 3991872
2   chromedriver                        0x0000000102fa2548 chromedriver + 189768
3   chromedriver                        0x0000000102fd0ef8 chromedriver + 380664
4   chromedriver                        0x0000000102fc73bc chromedriver + 340924
5   chromedriver                        0x0000000102ff9bfc chromedriver + 547836
6   chromedriver                        0x0000000102fc6248 chromedriver + 336456
7   chromedriver                        0x000000010336f548 chromedriver + 4175176
8   chromedriver                        0x0000000103385

---


## Bookie 3

In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import pickle

options = Options()
options.headless = True
options.add_argument('window-size=1920x1080')

web = 'https://www.betfair.com/sport/inplay'
path =  '/Users/jaredgodar/codeup-data-science/surebet/chromedriver'

driver = webdriver.Chrome(path, options=options)
driver.get(web)
# driver.maximize_window()

#option1
# accept = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')))
#option 2
time.sleep(2)
accept = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
accept.click()
dict_odds = {}

#choose any market you want
markets = ['Más/menos de 2,5 goles', '¿Marcarán ambos equipos?']

for i, market in enumerate(markets):
    dropdown = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'dropdown-1')))
    dropdown.click()
    chooser = WebDriverWait(dropdown, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),'+'"'+str(market)+'"'+')]')))
    chooser.click()
    list_odds = []
    teams = []
    time.sleep(1)
    box = driver.find_element_by_xpath('//div[contains(@data-sport-id,"1")]') #livebox -> diferent sports //  #upcoming = comingup
    rows = WebDriverWait(box, 5).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'com-coupon-line')))
    for row in rows:
        odds = row.find_element_by_xpath('.//div[contains(@class, "runner-list")]')
        list_odds.append(odds.text)
        home = row.find_element_by_class_name('home-team-name').text
        away = row.find_element_by_class_name('away-team-name').text
        teams.append(home + '\n' + away)
    dict_odds['odds_%s' % i] = list_odds
    dict_odds['teams_%s' % i] = teams

driver.quit()

# unlimited columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

df_1 = pd.DataFrame({'Teams':dict_odds['teams_0'], 'over2.5':dict_odds['odds_0']}).set_index('Teams')
df_btts = pd.DataFrame({'Teams':dict_odds['teams_1'], 'btts':dict_odds['odds_1']}).set_index('Teams')

df_betfair = pd.concat([df_1, df_btts], axis=1, sort=True)
df_betfair.reset_index(inplace=True)
df_betfair.rename(columns={'index':'Teams'}, inplace=True)

df_betfair = df_betfair.fillna('')
df_betfair = df_betfair.replace('SUSPENDIDO\n', '', regex=True)#Spanish
df_betfair = df_betfair.applymap(lambda x: x.strip() if isinstance(x, str) else x) #14.0\n

#save file
output = open('df_betfair', 'wb')
pickle.dump(df_betfair, output)
output.close()
print(df_betfair)

/var/folders/9y/17ptyrds267724n2_7s7hx340000gn/T/ipykernel_30941/2370651308.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path, options=options)
/var/folders/9y/17ptyrds267724n2_7s7hx340000gn/T/ipykernel_30941/2370651308.py:25: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  accept = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="onetrust-accept-btn-handler"]"}
  (Session info: headless chrome=99.0.4844.51)
Stacktrace:
0   chromedriver                        0x0000000100f4f370 chromedriver + 4420464
1   chromedriver                        0x0000000100ee6940 chromedriver + 3991872
2   chromedriver                        0x0000000100b46548 chromedriver + 189768
3   chromedriver                        0x0000000100b74ef8 chromedriver + 380664
4   chromedriver                        0x0000000100b9dbfc chromedriver + 547836
5   chromedriver                        0x0000000100b6a248 chromedriver + 336456
6   chromedriver                        0x0000000100f13548 chromedriver + 4175176
7   chromedriver                        0x0000000100f2995c chromedriver + 4266332
8   chromedriver                        0x0000000100f2e660 chromedriver + 4286048
9   chromedriver                        0x0000000100f2a124 c